In [1]:
from scipy.stats import linregress as LR
from ipywidgets import interact
import ipywidgets as widgets
from dataclasses import dataclass
from typing import Union, Tuple, Dict, List
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt

In [2]:
@dataclass
class HugoniotEOS:
    name: str
    rho0: float
    C0: float
    S: float
    def hugoniot_eos(self, up: Union[float, npt.ArrayLike]) -> Union[float, npt.ArrayLike] :
        '''
        Calculate Us from linear Us-Up relationship
        '''
        Us = self.C0 + self.S*up
        return Us

    def hugoniot_P(self, up:Union[float, npt.ArrayLike])->Union[float, npt.ArrayLike]:
        '''
        Calculate pressure by using linear Us-Up relationship to get Us from Up input. 
        '''
        Us = self.hugoniot_eos(up)
        P = self.rho0*Us*up
        return P
    def solve_up(self, P: Union[npt.ArrayLike, float]) -> Union[npt.ArrayLike, float]:
        '''
        Get the Up for a given pressure or range of pressures. By solving a quadratic equation
        and taking the positive root. 
        '''
        a = self.S
        b = self.C0
        c = -P/self.rho0

        Up = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
        return Up
@dataclass
class MixedHugoniotEOS(HugoniotEOS):
    components: List[str]
    vfracs: List[float]
    
def convert_volfrac_to_massfrac(rho1: Union[float, npt.ArrayLike], rho2: Union[float, npt.ArrayLike], Vx1: float) -> Union[float, npt.ArrayLike]:
    '''
    Converts volume frac to mass frac. 
    Expects Vx1 to be between 0 and 1
    will return the mass fraction for a given volume fraction of 
    the material with rho1. 
    '''
    mass_1 = rho1 * Vx1
    mass_2 = rho2 * (1-Vx1)
    x1 = mass_1/(mass_1 + mass_2)
    return x1
    
def generate_mixed_hugoniot_many(name:str,material_list: List[Tuple[HugoniotEOS, float]], Up: npt.ArrayLike  = np.linspace(0,8, 1000)) -> MixedHugoniotEOS:
    """
    Generates a mixed Hugoniot for a given list of materials and their volume percent

    :param name: name of new mixture
    :param material_list: list of tuples containing the material and it's respective volume fraction [(mat1, vx1), (mat2, vx2)]
    :param Up: array of particle velocities to solve for. This will be applied to the first material to get a P, after which
            We will solve for Up at the first materials pressures. 
    :returns: MixedHugonitEOS
    :raises ValueError: raises if sum of volume percents is not equal to 1
    """
    vols = [i[1] for i in material_list]
    if sum(vols) != 1:
        raise ValueError("Volume fractions must sum to 1")
    mat1, xv1 = material_list[0]
    P = mat1.hugoniot_P(Up)
    Up_list = [Up]
    masses = []
    vols = []
    names = []
    xvs = []
    masses.append(mat1.rho0 * xv1)
    vols.append(masses[0]/mat1.rho0)
    for mat, xv in material_list[1:]:
        names.append(mat.name)
        xvs.append(xv)
        mass = mat.rho0 * xv
        vol = mass /mat.rho0
        masses.append(mass)
        vols.append(vol)
        Up_list.append(mat.solve_up(P))
    rho_mix = sum(masses)/sum(vols)
    mass_frac = [m/sum(masses) for m in masses]
    mixed_Up = np.sqrt(sum([up**2  * x for up,x in zip(Up_list, mass_frac)]))
    mixed_Us = (P[1:])/(rho_mix*mixed_Up[1:])
    regression = LR(mixed_Up[1:], mixed_Us)
    
    mixed = MixedHugoniotEOS(name,rho_mix, regression.intercept, regression.slope, names, xvs )
    mixed.mfracs = mass_frac
    return mixed
            
        

def generate_mixed_hugoniot(name:str, material1: HugoniotEOS, material2: HugoniotEOS, Vx_mat1:float, Up: npt.ArrayLike = np.linspace(0,8, 1000)) -> MixedHugoniotEOS:
    """
    Generates a mixed Hugoniot for a given list of materials and their volume percent

    :param name: name of 
    :param param2: this is a second param
    :returns: this is a description of what is returned
    :raises keyError: raises an exception
    """
    P = material1.hugoniot_P(Up) 
    material1Up = Up
    material2Up = material2.solve_up(P)

    mass_mat1 = material1.rho0 * Vx_mat1
    mass_mat2 = material2.rho0 * (1-Vx_mat1)
    x_mat1 = convert_volfrac_to_massfrac(material1.rho0, material2.rho0, Vx_mat1)
    rho_mix = (mass_mat1 + mass_mat2)/((mass_mat1/material1.rho0) + (mass_mat2/material2.rho0))
    mixed_Up = np.sqrt(material1Up**2 * x_mat1 + material2Up**2 * (1-x_mat1))
    # avoid divide by zero warning, shortens result array by 1
    mixed_Us = (P[1:])/(rho_mix*mixed_Up[1:])
    regression = LR(mixed_Up[1:], mixed_Us) # match sizes of arrays by skipping first item in Up
    names = [material1.name, material2.name]
    vols = [Vx_mat1, 1-Vx_mat1]
    mfracs = [x_mat1, 1-x_mat1]
    mixed = MixedHugoniotEOS(name, rho_mix, regression.intercept, regression.slope, names, vols)
    mixed.mfracs = mfracs
    return mixed

def plot_mixture(material1: HugoniotEOS, material2: HugoniotEOS, volpercent:float) -> MixedHugoniotEOS:
    up1 = np.linspace(0.,6,1000)
    mat_list = [(material1, volpercent), (material2, 1-volpercent)]
    mix = generate_mixed_hugoniot(f'vol{str(volpercent)+material1.name + material2.name}', material1, material2, volpercent, up1)
    P = material1.hugoniot_P(up1)
    upmix = mix.solve_up(P)
    up2 = material2.solve_up(P)
    
    fig,ax = plt.subplots(1,2, figsize=(16,8))
    plt.rcParams.update({'font.size': 22})
    ax[0].plot(up1,P,'-b',linewidth=3,label=material1.name)
    ax[0].plot(material2.solve_up(P),P,'-r',linewidth=3,label=material2.name)
    ax[0].plot(mix.solve_up(P),P,'--m',linewidth=3,label=f'{mix.vfracs[0]*100:.1f} %v {material1.name}')
    ax[0].set_xlabel('up (km/s)') 
    ax[0].set_ylabel('P (GPa)')
    ax[0].legend(fontsize=14)

    ax[1].set_xlabel("Up")
    ax[1].set_ylabel("Us")
    ax[1].plot(up1, mix.hugoniot_eos(up1), label = f'{mix.vfracs[0]*100:.1f} %v {material1.name}')
    ax[1].plot(up1, material1.hugoniot_eos(up1), label  = material1.name)
    ax[1].plot(up1, material2.hugoniot_eos(up1), label = material2.name)
    ax[1].legend()
    return mix

In [3]:
# Create interactive widgets for material 1
from ipywidgets.widgets import VBox, HBox
import pprint
from time import sleep
title1 = widgets.HTML('<h1>Material 1</h1>')
name1 = widgets.Text(value="MgO", description="Name")
rho0_1 = widgets.FloatText(value=3.583, description="Density")
C0_1 = widgets.FloatText(value=6.661, description="C0")
S_1 = widgets.FloatText(value=1.36, description="S")
material1_box = VBox([title1, name1, rho0_1, C0_1, S_1])

# Create interactive widgets for material 2
title2 = widgets.HTML('<h1>Material 2</h1>')
name2 = widgets.Text(value="Epoxy", description="Name")
rho0_2 = widgets.FloatText(value=1.2, description="Density")
C0_2 = widgets.FloatText(value=2.9443, descripton="C0")
S_2 = widgets.FloatText(value=1.3395, description="S")
material2_box = VBox([title2, name2, rho0_2, C0_2, S_2])

# Create widget for volume percentage
volpercent = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, step=0.01, description='Mat1 v1/v')
# Combine all widgets into a single layout
ui = HBox([material1_box, material2_box, ])
ui = VBox([ui, volpercent])
def update_plot(name1, rho0_1, C0_1, S_1, name2, rho0_2, C0_2, S_2, volpercent):
    material1 = HugoniotEOS(name=name1, rho0=rho0_1, C0=C0_1, S=S_1)
    material2 = HugoniotEOS(name=name2, rho0=rho0_2, C0=C0_2, S=S_2)
    mix =  plot_mixture(material1, material2, volpercent)
    print("*"*50)
    print(f'Mixture parameters\n Density: {mix.rho0}\n C0: {mix.C0}\n S: {mix.S}')
    print(f'Weight percent {material1.name}: {mix.mfracs[0]}')
    print('The mixed hugoniot data is defined in this notebook as the variable "mix"')
out = widgets.interactive_output(update_plot, {'name1': name1, 'rho0_1': rho0_1, 'C0_1': C0_1, 'S_1': S_1, 'name2': name2, 'rho0_2': rho0_2, 'C0_2': C0_2, 'S_2': S_2, 'volpercent': volpercent})


display(ui, out)

Output()

In [4]:
def plot_mixture(material1: HugoniotEOS, material2: HugoniotEOS, volpercent:float) -> MixedHugoniotEOS:
    up1 = np.linspace(0.,6,1000)
    mat_list = [(material1, volpercent), (material2, 1-volpercent)]
    mix = generate_mixed_hugoniot_many(f'vol{str(volpercent)+material1.name + material2.name}', mat_list, up1)
    P = material1.hugoniot_P(up1)
    upmix = mix.solve_up(P)
    up2 = material2.solve_up(P)
    
    fig,ax = plt.subplots(1,2, figsize=(16,8))
    plt.rcParams.update({'font.size': 22})
    ax[0].plot(up1,P,'-b',linewidth=3,label=material1.name)
    ax[0].plot(material2.solve_up(P),P,'-r',linewidth=3,label=material2.name)
    ax[0].plot(mix.solve_up(P),P,'--m',linewidth=3,label=f'{mix.vfracs[0]*100:.1f} %v {material1.name}')
    ax[0].set_xlabel('up (km/s)') 
    ax[0].set_ylabel('P (GPa)')
    ax[0].legend(fontsize=14)

    ax[1].set_xlabel("Up")
    ax[1].set_ylabel("Us")
    ax[1].plot(up1, mix.hugoniot_eos(up1), label = f'{mix.vfracs[0]*100:.1f} %v {material1.name}')
    ax[1].plot(up1, material1.hugoniot_eos(up1), label  = material1.name)
    ax[1].plot(up1, material2.hugoniot_eos(up1), label = material2.name)
    ax[1].legend()
    return mix
from ipywidgets.widgets import VBox, HBox
import pprint
from time import sleep
title1 = widgets.HTML('<h1>Component 1</h1>')
name1 = widgets.Text(value="MgO", description="Name")
rho0_1 = widgets.FloatText(value=3.583, description="Density")
C0_1 = widgets.FloatText(value=6.661, description="C0")
S_1 = widgets.FloatText(value=1.36, description="S")
material1_box = VBox([title1, name1, rho0_1, C0_1, S_1])

# Create interactive widgets for material 2
title2 = widgets.HTML('<h1>Component 2</h1>')
name2 = widgets.Text(value="Epoxy", description="Name")
rho0_2 = widgets.FloatText(value=1.2, description="Density")
C0_2 = widgets.FloatText(value=2.9443, descripton="C0")
S_2 = widgets.FloatText(value=1.3395, description="S")
material2_box = VBox([title2, name2, rho0_2, C0_2, S_2])

# Create widget for volume percentage
volpercent = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, step=0.01, description='Mat1 v1/v')
# Combine all widgets into a single layout
ui = HBox([material1_box, material2_box, ])
ui = VBox([ui, volpercent])
def update_plot(name1, rho0_1, C0_1, S_1, name2, rho0_2, C0_2, S_2, volpercent):
    material1 = HugoniotEOS(name=name1, rho0=rho0_1, C0=C0_1, S=S_1)
    material2 = HugoniotEOS(name=name2, rho0=rho0_2, C0=C0_2, S=S_2)
    mix =  plot_mixture(material1, material2, volpercent)
    print("*"*50)
    print(f'Mixture parameters\n Density: {mix.rho0}\n C0: {mix.C0}\n S: {mix.S}')
    print(f'Weight percent {material1.name}: {mix.mfracs[0]}')
    print('The mixed hugoniot data is defined in this notebook as the variable "mix"')
out = widgets.interactive_output(update_plot, {'name1': name1, 'rho0_1': rho0_1, 'C0_1': C0_1, 'S_1': S_1, 'name2': name2, 'rho0_2': rho0_2, 'C0_2': C0_2, 'S_2': S_2, 'volpercent': volpercent})


display(ui, out)

Output()